# Text generation with an RNN

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
from google.colab import files
file=files.upload()
!mkdir /root/.kaggle
!mv kaggle.json  /root/.kaggle


Saving kaggle.json to kaggle.json


In [4]:
#!kaggle datasets download -d nzalake52/new-york-times-articles
!kaggle datasets download -d ad6398/bbcnewsarticle
!unzip bbcnewsarticle.zip

  0% 0.00/1.83M [00:00<?, ?B/s]
100% 1.83M/1.83M [00:00<00:00, 61.6MB/s]
Archive:  bbcnewsarticle.zip
  inflating: bbc-text.csv            


In [5]:
# Read, then decode for py2 compat.
#text = open("shakespeare-text.txt").read()
text=open("bbc-text.csv",encoding='utf8').read()
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
#print(sequences)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

#dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

print(dataset)
for d in dataset.take(1):
    print(d[0][0,3].numpy())
    #print(d[0])
    #print(d[1])

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>
36


## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [8]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [9]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [10]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model.summary()
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15104     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 59)            60475     
Total params: 5,322,555
Trainable params: 5,322,555
Non-trainable params: 0
_________________________________________________________________


In [12]:

EPOCHS = 60
try:
  model.load_weights("weights.h5")
except:
  pass
history = model.fit(dataset, epochs=EPOCHS)
model.save_weights("weights.h5")

Epoch 1/60
782/782 [==============================] - 43s 55ms/step - loss: 1.9793
Epoch 2/60
782/782 [==============================] - 42s 54ms/step - loss: 1.4290
Epoch 3/60
782/782 [==============================] - 42s 54ms/step - loss: 1.3156
Epoch 4/60
782/782 [==============================] - 43s 54ms/step - loss: 1.2520
Epoch 5/60
782/782 [==============================] - 42s 54ms/step - loss: 1.2040
Epoch 6/60
782/782 [==============================] - 42s 54ms/step - loss: 1.1632
Epoch 7/60
782/782 [==============================] - 42s 54ms/step - loss: 1.1271
Epoch 8/60
782/782 [==============================] - 42s 54ms/step - loss: 1.0958
Epoch 9/60
782/782 [==============================] - 42s 54ms/step - loss: 1.0702
Epoch 10/60
782/782 [==============================] - 42s 54ms/step - loss: 1.0481
Epoch 11/60
782/782 [==============================] - 42s 54ms/step - loss: 1.0286
Epoch 12/60
782/782 [==============================] - 42s 54ms/step - loss: 1.0129
E

## Generate text

In [13]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights("weights.h5")
#model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            15104     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 59)             60475     
Total params: 5,322,555
Trainable params: 5,322,555
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* Begin by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [14]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [19]:
print(generate_text(model, start_string=u"telegraph"))

telegraphies are yet.  this got a lot ot so far  says civil  including rock schw  the shadow chancellor wandeated profits to companies to swap off their responsibility - for mobile owners have already grappide any phone structure  according to the service at net science. the film is the usa  psais  mci ambas s milan iselected by other people in the finest hound.  the 10-year-old  on thursday  mr thomas. it also officially credible leads that compared to 140 000 time s today at renan.  i would have told a service people have nothing to look at  with a string owner further expirational evidence of disguises to copber jobs when (9-1. peter  alex ferguson. sie starring copyconner  if the wise is aim of teacher it would ever say.  the scissor roof antolina detention is lightly due calen to 2.5%. chelsea  perior  who usata is due to launch new devon the frenchmand helmily so far. this year they will rise to a target of a double economy as a representation.  on wednesday from rise. marathon y

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.